### Import User_defined_Notebook into Gold_Layer

In [0]:
%run "/Workspace/Users/rajeshc@maveric-systems.com/Capstone project/User_Defined_Functions"

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql.window import Window
from datetime import datetime, timedelta
import pandas as pd
 

In [0]:
# Read the table "cleaned_transformed_cust" from the "silver_schema" database
gold_cust_df = spark.read.table("silver_schema.cleaned_transformed_cust")


In [0]:
# Read the table "cleaned_transformed_branch" from the "silver_schema" database
gold_branch_df = spark.read.table("silver_schema.cleaned_transformed_branch")

In [0]:
# Read the table "cleaned_transformed_trans" from the "silver_schema" database
gold_trans_df = spark.read.table("silver_schema.cleaned_transformed_trans")

## Aggregation

# Creating Segment Table

In [0]:
# Fixed current date
current_date = pd.Timestamp("2023-06-01")

# High_Value: Customers with high transaction volume
high_value_threshold = 10000
high_value_customers = gold_trans_df.groupBy("customer_id").agg(
    sum("amount_usd").alias("total_amount")
)
high_value_customers = (
    high_value_customers.filter(col("total_amount") > high_value_threshold)
    .select("customer_id")
    .withColumn("segment_name", lit("High_Value"))
    .withColumn("segment_description", lit("Customers with high transaction volume"))
    .withColumn("last_update", lit(current_date))
)

# New_User: Customers who joined in the last 30 days
new_user_customers = (
    gold_cust_df.filter(col("join_date") > current_date - timedelta(days=30))
    .select("customer_id")
    .withColumn("segment_name", lit("New_User"))
    .withColumn("segment_description", lit("Customers who joined in last 30 days"))
    .withColumn("last_update", lit(current_date))
)

# Inactive: Customers with no transactions in last 90 days
recent_transactions = (
    gold_trans_df.filter(
        col("transaction_datetime") > current_date - timedelta(days=90)
    )
    .select("customer_id")
    .distinct()
)
inactive_customers = (
    gold_cust_df.join(recent_transactions, on="customer_id", how="left_anti")
    .select("customer_id")
    .withColumn("segment_name", lit("Inactive"))
    .withColumn("segment_description", lit("No transactions in last 90 days"))
    .withColumn("last_update", lit(current_date))
)

# Credit_Risk: Customers with low credit scores (assuming credit_score < 600)
credit_risk_customers = (
    gold_cust_df.filter(col("credit_score") < 600)
    .select("customer_id")
    .withColumn("segment_name", lit("Credit_Risk"))
    .withColumn("segment_description", lit("Customers with low credit scores"))
    .withColumn("last_update", lit(current_date))
)

# Loyal: Customers with consistent activity for over 5 years
loyal_customers = (
    gold_cust_df.filter(col("join_date") < current_date - timedelta(days=5 * 365))
    .select("customer_id")
    .withColumn("segment_name", lit("Loyal"))
    .withColumn("segment_description", lit("Consistent activity for over 5 years"))
    .withColumn("last_update", lit(current_date))
)

# Combine all segments
customer_segmentation_df = (
    high_value_customers.union(new_user_customers)
    .union(inactive_customers)
    .union(credit_risk_customers)
    .union(loyal_customers)
)

# Add segment_id
window_spec = Window.orderBy("customer_id")
customer_segmentation_df = customer_segmentation_df.withColumn(
    "segment_id", row_number().over(window_spec)
)

# Show the result
customer_segmentation_df = customer_segmentation_df.select(
    "segment_id", "customer_id", "segment_name", "segment_description", "last_update"
)

customer_segmentation_df.display()

segment_id,customer_id,segment_name,segment_description,last_update
1,C1000,High_Value,Customers with high transaction volume,2023-06-01T00:00:00Z
2,C1000,Loyal,Consistent activity for over 5 years,2023-06-01T00:00:00Z
3,C1001,High_Value,Customers with high transaction volume,2023-06-01T00:00:00Z
4,C1001,Loyal,Consistent activity for over 5 years,2023-06-01T00:00:00Z
5,C1002,High_Value,Customers with high transaction volume,2023-06-01T00:00:00Z
6,C1002,Loyal,Consistent activity for over 5 years,2023-06-01T00:00:00Z
7,C1003,High_Value,Customers with high transaction volume,2023-06-01T00:00:00Z
8,C1003,Loyal,Consistent activity for over 5 years,2023-06-01T00:00:00Z
9,C1004,High_Value,Customers with high transaction volume,2023-06-01T00:00:00Z
10,C1004,Loyal,Consistent activity for over 5 years,2023-06-01T00:00:00Z


In [0]:
# Analyzing the customer_segmentation DataFrame for statistics like :
# Number of rows, Column names, Distinct counts for each column,Data types, Null count, Duplicate count

customer_segmentation_basic_info=analyze_dataframe(customer_segmentation_df)


Number of rows: 1305
Number of columns: 5

Column names: ['segment_id', 'customer_id', 'segment_name', 'segment_description', 'last_update']

Distinct counts for each column:
segment_id: 1305
customer_id: 847
segment_name: 4
segment_description: 4
last_update: 1

Data types:
segment_id: IntegerType()
customer_id: StringType()
segment_name: StringType()
segment_description: StringType()
last_update: TimestampType()

Null values count and % Null values:
segment_id: 0 (0.00%)
customer_id: 0 (0.00%)
segment_name: 0 (0.00%)
segment_description: 0 (0.00%)
last_update: 0 (0.00%)

Duplicate Data Details:
No duplicate rows found.


In [0]:
# Group by "segment_name" and aggregate the count of each segment
segment_count_df = customer_segmentation_df.groupBy("segment_name").agg(count("segment_name").alias("count"))
 
segment_count_df.show()

+------------+-----+
|segment_name|count|
+------------+-----+
|  High_Value|  781|
|    Inactive|    4|
| Credit_Risk|  276|
|       Loyal|  244|
+------------+-----+



In [0]:
customer_segmentation_df.select("segment_name").distinct().show()

+------------+
|segment_name|
+------------+
|  High_Value|
|    Inactive|
| Credit_Risk|
|       Loyal|
+------------+



##Fraud Flag

In [0]:
from pyspark.sql.functions import col, lit, when, concat, lpad, row_number
from pyspark.sql.window import Window

# Function to detect unusual amounts
def detect_unusual_amount(df):
    return df.withColumn(
        "unusual_amount",
        when(col("amount_usd") > 50000, lit(0.75)).otherwise(lit(None))
    )

# Function to detect velocity of transactions
def detect_velocity_check(df):
    monthly_counts = df.groupBy("year", "month", "customer_id") \
        .agg(count("transaction_id").alias("monthly_count"))
    
    df_with_counts = df.join(monthly_counts, on=["month", "year", "customer_id"], how="left")
    
    return df_with_counts.withColumn(
        "velocity_check",
        when(col("monthly_count") > 7, lit(0.60)).otherwise(lit(None))
    )

# Function to detect watchlist matches
def detect_watchlist_match(df):
    return df.withColumn(
        "watchlist_match",
        when(col("amount_usd") > 30000, lit(0.90)).otherwise(lit(None))
    )

# Function to detect pattern anomalies
def detect_pattern_anomaly(df):
    return df.withColumn(
        "pattern_anomaly",
        when(col("amount_usd") > 100000, lit(0.85)).otherwise(lit(None))
    )

# Apply fraud detection functions to the DataFrame
transaction_df = detect_unusual_amount(gold_trans_df)
transaction_df = detect_velocity_check(transaction_df)
transaction_df = detect_watchlist_match(transaction_df)
transaction_df = detect_pattern_anomaly(transaction_df)

# Combine fraud flags into a single column and set the confidence score
fraud_flags_df = transaction_df.select(
    col("transaction_id"),
    col("customer_id"),
    col("month"),
    col("year"),
    when(col("unusual_amount").isNotNull(), lit("unusual_amount"))
    .when(col("velocity_check").isNotNull(), lit("velocity_check"))
    .when(col("watchlist_match").isNotNull(), lit("watchlist_match"))
    .when(col("pattern_anomaly").isNotNull(), lit("pattern_anomaly"))
    .alias("flag_type"),
    when(col("unusual_amount").isNotNull(), col("unusual_amount"))
    .when(col("velocity_check").isNotNull(), col("velocity_check"))
    .when(col("watchlist_match").isNotNull(), col("watchlist_match"))
    .when(col("pattern_anomaly").isNotNull(), col("pattern_anomaly"))
    .alias("confidence_score"),
    col("transaction_datetime")
).filter(col("flag_type").isNotNull())

# Add flag_id to the DataFrame
window_spec = Window.orderBy("transaction_datetime")

fraud_flags_df = fraud_flags_df.withColumn("flag_id",
    concat(lit("F"), lpad(row_number().over(window_spec).cast("string"), 4, "0"))
)

# Display the final DataFrame with fraud flags
fraud_flags_df.display()


transaction_id,customer_id,month,year,flag_type,confidence_score,transaction_datetime,flag_id
T5004,C1102,Jan,2018,velocity_check,0.6,2018-01-01T05:14:00Z,F0001
T5010,C1108,Jan,2018,velocity_check,0.6,2018-01-01T12:12:00Z,F0002
T5012,C1040,Jan,2018,unusual_amount,0.75,2018-01-01T14:31:00Z,F0003
T5015,C1051,Jan,2018,velocity_check,0.6,2018-01-01T17:06:00Z,F0004
T5019,C1034,Jan,2018,velocity_check,0.6,2018-01-01T20:56:00Z,F0005
T5021,C1102,Jan,2018,unusual_amount,0.75,2018-01-01T23:07:00Z,F0006
T5022,C1174,Jan,2018,watchlist_match,0.9,2018-01-01T23:48:00Z,F0007
T5035,C1039,Jan,2018,watchlist_match,0.9,2018-01-02T12:28:00Z,F0008
T5042,C1075,Jan,2018,velocity_check,0.6,2018-01-02T20:04:00Z,F0009
T5051,C1075,Jan,2018,velocity_check,0.6,2018-01-03T06:25:00Z,F0010


In [0]:
fraud_flags_df.filter("flag_type = 'watchlist_match'").count()

463

In [0]:
customer_segment_counts = customer_segmentation_df.groupBy("segment_name").count()
customer_segment_counts.display()

segment_name,count
High_Value,781
Inactive,4
Credit_Risk,276
Loyal,244


In [0]:
segment_details = customer_segmentation_df.groupBy("segment_name").agg(
    countDistinct("customer_id").alias("customer_count"),
    first("segment_description").alias("segment_description"),
    first("last_update").alias("last_update")
)


segment_details.display()

segment_name,customer_count,segment_description,last_update
Credit_Risk,276,Customers with low credit scores,2023-06-01T00:00:00Z
High_Value,781,Customers with high transaction volume,2023-06-01T00:00:00Z
Inactive,4,No transactions in last 90 days,2023-06-01T00:00:00Z
Loyal,244,Consistent activity for over 5 years,2023-06-01T00:00:00Z


In [0]:
last_update_by_segment = customer_segmentation_df.groupBy("segment_name").agg(
    max("last_update").alias("last_update")
)
last_update_by_segment.display()

segment_name,last_update
High_Value,2023-06-01T00:00:00Z
Inactive,2023-06-01T00:00:00Z
Credit_Risk,2023-06-01T00:00:00Z
Loyal,2023-06-01T00:00:00Z


In [0]:
segment_transaction_summary = (
    gold_trans_df
    .join(customer_segmentation_df, on="customer_id", how="inner")
    .groupBy("segment_name")
    .agg(
        avg("amount_usd").alias("average_transaction_amount"),
        sum("amount_usd").alias("total_transaction_amount")
    )
)
segment_transaction_summary.display()

segment_name,average_transaction_amount,total_transaction_amount
High_Value,3468.506516500417,1.488266776099999E8
Inactive,4405.336136363636,387669.57999999996
Credit_Risk,3253.9712336595435,4.555234329999995E7
Loyal,3085.5695385824783,6.673778355000042E7


In [0]:
segment_transaction_count = (
    gold_trans_df
    .join(customer_segmentation_df, on="customer_id", how="inner")
    .groupBy("segment_name")
    .agg(
        count("transaction_id").alias("transaction_count")
    )
)
segment_transaction_count.display()

segment_name,transaction_count
High_Value,42908
Inactive,88
Credit_Risk,13999
Loyal,21629


In [0]:
segment_total_transaction_amount = (
    gold_trans_df
    .join(customer_segmentation_df, on="customer_id", how="inner")
    .groupBy("segment_name")
    .agg(
        sum("amount_usd").alias("total_transaction_amount")
    )
)
segment_total_transaction_amount.display()

segment_name,total_transaction_amount
High_Value,1.488266776099999E8
Inactive,387669.57999999996
Credit_Risk,4.555234329999995E7
Loyal,6.673778355000042E7


In [0]:

# Corrected code without the incorrect retention rate calculation
retention_analysis = (
    gold_cust_df
    .join(customer_segmentation_df, on="customer_id", how="inner")
    .groupBy("segment_name")
    .agg(
        countDistinct("customer_id").alias("active_customer_count")
    )
)

retention_analysis.display()

segment_name,active_customer_count
High_Value,781
Loyal,244
Credit_Risk,276
Inactive,4


In [0]:
monthly_summary = (gold_trans_df
                   .groupBy("year","month","customer_id")
                   .agg(
                       count("transaction_id").alias("transaction_count"),
                       round(sum("amount_usd"),2).alias("total_transaction_amount")
                   )).orderBy("year","month", "customer_id")

monthly_summary.display()

year,month,customer_id,transaction_count,total_transaction_amount
2018,Apr,C1000,2,55.11
2018,Apr,C1002,4,207.89
2018,Apr,C1003,4,222.4
2018,Apr,C1004,4,121.71
2018,Apr,C1005,8,373.02
2018,Apr,C1006,6,226.13
2018,Apr,C1007,4,361.2
2018,Apr,C1008,3,56.28
2018,Apr,C1009,3,154.48
2018,Apr,C1010,2,124.0


In [0]:
customer_transaction_counts = (gold_trans_df
                               .groupBy("customer_id")
                               .agg(count("transaction_id").alias("transaction_count")))

# Show the result
customer_transaction_counts.display()

customer_id,transaction_count
C1100,87
C1804,23
C1571,34
C1524,39
C1842,12
C1602,38
C1875,9
C1628,41
C1305,62
C1829,23


In [0]:
# Aggregations by Branch with Amounts
branch_aggregations_df = gold_trans_df.groupBy("branch_id").agg(
    round(sum("Amount_USD"), 2).alias("total_amount"),
    round(avg("Amount_USD"), 2).alias("average_amount"),
    min("Amount_USD").alias("min_amount"),
    max("Amount_USD").alias("max_amount"),
    count("Amount_USD").alias("transaction_count")
)

branch_aggregations_df.display()

branch_id,total_amount,average_amount,min_amount,max_amount,transaction_count
B0014,7559274.94,2410.48,0.75,74789.86,3136
B0004,1.002818039E7,3168.46,1.1,108970.0,3165
B0010,1.130075077E7,3584.13,1.3,129705.23,3153
B0006,1.00112328E7,3256.74,1.1,109433.61,3074
B0012,1.170618094E7,3623.08,1.1,109044.0,3231
B0013,8583319.53,2719.68,1.0,99092.28,3156
B0001,8589440.57,2618.73,1.0,99922.3,3280
B0008,1.237532113E7,4001.07,1.3,129509.32,3093
B0007,9085222.33,2878.71,1.1,109567.16,3156
B0003,1.014738145E7,3135.78,1.0,99961.89,3236


In [0]:
customer_aggregations_df = gold_trans_df.groupBy("customer_id").agg(
    round(sum("Amount_USD"), 2).alias("total_amount"),
    round(avg("Amount_USD"), 2).alias("average_amount"),
    min("Amount_USD").alias("min_amount"),
    max("Amount_USD").alias("max_amount"),
    count("Amount_USD").alias("transaction_count")
)

customer_aggregations_df.display()

customer_id,total_amount,average_amount,min_amount,max_amount,transaction_count
C1100,137492.11,1580.37,1.1,77429.27,87
C1804,1020.89,44.39,1.66,229.4,23
C1571,192798.58,5670.55,2.03,102543.77,34
C1524,119288.47,3058.68,1.1,103950.17,39
C1842,498.42,41.54,1.0,106.29,12
C1602,179173.59,4715.09,1.0,75139.77,38
C1875,629.43,69.94,5.42,142.38,9
C1628,80673.98,1967.66,1.0,78918.52,41
C1305,315427.63,5087.54,1.47,96367.93,62
C1829,1151.08,50.05,1.1,140.47,23


In [0]:
transaction_with_location_df = gold_trans_df.join(
    gold_branch_df.select("branch_id", "location", "timezone"),
    on="branch_id",
    how="left"
)


# Aggregations by Location with Rounded Amounts
location_aggregations_df = transaction_with_location_df.groupBy("location").agg(
    round(sum("Amount_USD"), 2).alias("total_amount"),
    round(avg("Amount_USD"), 2).alias("average_amount"),
    min("Amount_USD").alias("min_amount"),
    max("Amount_USD").alias("max_amount"),
    count("Amount_USD").alias("transaction_count")
)

location_aggregations_df.display()

location,total_amount,average_amount,min_amount,max_amount,transaction_count
Phoenix,9085222.33,2878.71,1.1,109567.16,3156
Madrid,1.002818039E7,3168.46,1.1,108970.0,3165
Dallas,1.014738145E7,3135.78,1.0,99961.89,3236
Philadelphia,1.144304857E7,3555.95,1.3,129853.26,3218
Los Angeles,1.00112328E7,3256.74,1.1,109433.61,3074
San Diego,7559274.94,2410.48,0.75,74789.86,3136
London,8583319.53,2719.68,1.0,99092.28,3156
Brisbane,9457705.3,3069.69,1.1,109241.83,3081
Perth,8589440.57,2618.73,1.0,99922.3,3280
Paris,1.170618094E7,3623.08,1.1,109044.0,3231


In [0]:
timezone_aggregations_df = transaction_with_location_df.groupBy("timezone").agg(
    round(sum("Amount_USD"), 2).alias("total_amount"),
    round(avg("Amount_USD"), 2).alias("average_amount"),
    min("Amount_USD").alias("min_amount"),
    max("Amount_USD").alias("max_amount"),
    count("Amount_USD").alias("transaction_count")
)

timezone_aggregations_df.display()

timezone,total_amount,average_amount,min_amount,max_amount,transaction_count
AWST,8589440.57,2618.73,1.0,99922.3,3280
CST,2.723060717E7,2836.52,1.0,99961.89,9600
CET,3.340384193E7,3486.1,1.1,129776.35,9582
MST,9085222.33,2878.71,1.1,109567.16,3156
PST,2.887125851E7,3083.55,0.75,129705.23,9363
GMT,8583319.53,2719.68,1.0,99092.28,3156
EST,1.144304857E7,3555.95,1.3,129853.26,3218
AEST,2.183302643E7,3536.29,1.1,129509.32,6174


In [0]:
# Aggregations by Currency with Amounts
currency_aggregations_df = gold_trans_df.groupBy("currency").agg(
    round(sum("Amount_USD"), 2).alias("total_amount"),
    round(avg("Amount_USD"), 2).alias("average_amount"),
    min("Amount_USD").alias("min_amount"),
    max("Amount_USD").alias("max_amount"),
    count("Amount_USD").alias("transaction_count")
)

currency_aggregations_df.display()


currency,total_amount,average_amount,min_amount,max_amount,transaction_count
GBP,4.678860107E7,3698.7,1.3,129853.26,12650
EUR,5.028852176E7,3201.66,1.1,109567.16,15707
AUD,7559274.94,2410.48,0.75,74789.86,3136
USD,4.440336727E7,2768.98,1.0,99961.89,16036


## Transaction table aggregations

In [0]:
channel_aggregations_df = gold_trans_df.groupBy("channel").agg(
    round(sum("Amount_USD"), 2).alias("total_amount"),
    round(avg("Amount_USD"), 2).alias("average_amount"),
    min("Amount_USD").alias("min_amount"),
    max("Amount_USD").alias("max_amount"),
    count("Amount_USD").alias("transaction_count")
)
display(channel_aggregations_df)

channel,total_amount,average_amount,min_amount,max_amount,transaction_count
MOBILE,4.592919653E7,3113.0,0.75,129853.26,14754
ATM,4.713586233E7,3159.03,0.75,128276.46,14921
BRANCH,8892493.0,3037.05,0.75,128861.12,2928
WEB,4.708221318E7,3154.38,0.75,129724.3,14926


In [0]:
# Aggregations by Transaction Type with Rounded Amounts
transaction_type_aggregations_df = gold_trans_df.groupBy("transaction_type").agg(
    round(sum("Amount_USD"), 2).alias("total_amount"),
    round(avg("Amount_USD"), 2).alias("average_amount"),
    min("Amount_USD").alias("min_amount"),
    max("Amount_USD").alias("max_amount"),
    count("Amount_USD").alias("transaction_count")
)

transaction_type_aggregations_df.display()


transaction_type,total_amount,average_amount,min_amount,max_amount,transaction_count
TRANSFER,1.988829446E7,3288.41,0.75,129853.26,6048
DEPOSIT,3.52281887E7,2981.65,0.75,129776.35,11815
WITHDRAWAL,5.503769904E7,3068.9,0.75,128928.01,17934
PAYMENT,3.888558284E7,3314.49,0.75,128816.16,11732


In [0]:
# Aggregations by Year and Month with Amounts
year_month_aggregations_df = gold_trans_df.groupBy("year", "month").agg(
    round(sum("Amount_USD"), 2).alias("total_amount"),
    round(avg("Amount_USD"), 2).alias("average_amount"),
    min("Amount_USD").alias("min_amount"),
    max("Amount_USD").alias("max_amount"),
    count("Amount_USD").alias("transaction_count")
)

year_month_aggregations_df.display()


year,month,total_amount,average_amount,min_amount,max_amount,transaction_count
2020,Mar,1848666.81,2759.2,1.0,119780.1,670
2022,Jun,2337289.92,3498.94,1.0,118957.63,668
2023,Feb,1750233.17,2813.88,0.75,120745.27,622
2018,Nov,2603537.22,3828.73,0.75,124023.86,680
2019,Jan,2250624.16,3215.18,1.0,120847.43,700
2021,Apr,2146771.64,3180.4,0.75,129187.53,675
2023,Aug,1862599.84,2672.31,0.75,97873.98,697
2021,Jan,2821925.51,4037.09,0.75,119460.99,699
2019,Jul,2870042.22,4171.57,0.78,124259.5,688
2019,Sep,1869882.08,2782.56,0.75,129705.23,672


In [0]:
# Aggregations by Status with Rounded Amounts
status_aggregations_df = gold_trans_df.groupBy("status").agg(
    round(sum("Amount_USD"), 2).alias("total_amount"),
    round(avg("Amount_USD"), 2).alias("average_amount"),
    min("Amount_USD").alias("min_amount"),
    max("Amount_USD").alias("max_amount"),
    count("Amount_USD").alias("transaction_count")
)

status_aggregations_df.display()


status,total_amount,average_amount,min_amount,max_amount,transaction_count
completed,1.2594154639E8,3120.07,0.75,129776.35,40365
denied,7284183.82,2985.32,0.75,126234.07,2440
pending,1.581403483E7,3347.59,0.75,129853.26,4724


In [0]:
# Aggregations by Channel and Transaction Type with Rounded Amounts
channel_type_aggregations_df = gold_trans_df.groupBy("channel", "transaction_type").agg(
    round(sum("Amount_USD"), 2).alias("total_amount"),
    round(avg("Amount_USD"), 2).alias("average_amount"),
    min("Amount_USD").alias("min_amount"),
    max("Amount_USD").alias("max_amount"),
    count("Amount_USD").alias("transaction_count")
)

channel_type_aggregations_df.display()


channel,transaction_type,total_amount,average_amount,min_amount,max_amount,transaction_count
BRANCH,TRANSFER,2939033.18,3139.99,0.75,128861.12,936
MOBILE,PAYMENT,1.214233105E7,3233.64,0.75,128816.16,3755
WEB,TRANSFER,5571115.37,3220.3,0.75,129724.3,1730
ATM,PAYMENT,1.302673394E7,3473.8,0.75,124966.54,3750
MOBILE,WITHDRAWAL,1.737855503E7,3088.97,0.75,127217.21,5626
WEB,PAYMENT,1.207884617E7,3255.75,0.75,121638.1,3710
MOBILE,DEPOSIT,1.074932961E7,2878.77,0.75,129776.35,3734
WEB,WITHDRAWAL,1.717632089E7,2989.27,0.75,128928.01,5746
WEB,DEPOSIT,1.225593075E7,3276.99,0.75,129705.23,3740
BRANCH,PAYMENT,1637671.68,3167.64,1.0,106993.49,517


In [0]:
# Define bins for transaction amount ranges
bins = [0, 50, 100, 500, 1000, 5000, 10000, float('inf')]
labels = ['0-50', '50-100', '100-500', '500-1000', '1000-5000', '5000-10000', '10000+']

# Create a new column for amount range
from pyspark.sql.functions import expr

transaction_frequency_df = gold_trans_df.withColumn(
    "amount_range",
    expr(f"CASE "
         f"WHEN Amount_USD <= {bins[1]} THEN '{labels[0]}' "
         f"WHEN Amount_USD <= {bins[2]} THEN '{labels[1]}' "
         f"WHEN Amount_USD <= {bins[3]} THEN '{labels[2]}' "
         f"WHEN Amount_USD <= {bins[4]} THEN '{labels[3]}' "
         f"WHEN Amount_USD <= {bins[5]} THEN '{labels[4]}' "
         f"WHEN Amount_USD <= {bins[6]} THEN '{labels[5]}' "
         f"ELSE '{labels[6]}' "  # This line replaces the check against float('inf')
         f"END AS amount_range")
).groupBy("amount_range").count().alias("transaction_count") 

transaction_frequency_df.display()


amount_range,count
5000-10000,5
0-50,27758
50-100,13385
100-500,4001
10000+,2380


In [0]:
# Calculate total amount USD
total_amount = gold_trans_df.agg(sum("Amount_USD")).collect()[0][0]

# Calculate percentage of total amount by channel
percentage_df = gold_trans_df.groupBy("channel").agg(
    round(sum("Amount_USD"), 2).alias("total_amount_usd")
).withColumn("percentage_of_total", round((col("total_amount_usd") / total_amount) * 100, 2))

percentage_df.show()


+-------+----------------+-------------------+
|channel|total_amount_usd|percentage_of_total|
+-------+----------------+-------------------+
| MOBILE|   4.592919653E7|              30.82|
|    ATM|   4.713586233E7|              31.63|
| BRANCH|       8892493.0|               5.97|
|    WEB|   4.708221318E7|              31.59|
+-------+----------------+-------------------+



In [0]:
currency_analysis = gold_trans_df.groupBy("currency") \
        .agg(
            sum("Amount_USD").alias("total_amount_by_currency"),
            avg("Amount_USD").alias("avg_amount_by_currency")
        )

currency_analysis.display()


currency,total_amount_by_currency,avg_amount_by_currency
GBP,4.67886010700002E7,3698.7036418972493
EUR,5.0288521760000095E7,3201.663064875539
AUD,7559274.939999998,2410.483080357142
USD,4.440336727000001E7,2768.9802488151668


In [0]:
excluded_dataframes = ["customer_df", "branch_df", "transaction_df", "df_with_cumulative_sum",
                       "loyal_customers", "credit_risk_customers", "inactive_customers",
                       "recent_transactions", "new_user_customers", "high_value_customers"]

# Filter out the excluded DataFrames
dataframes_to_save = [(df, name) for name, df in dataframes.items() if name not in excluded_dataframes]

# Save each remaining DataFrame to a table
for df, name in dataframes_to_save:
    table_name = f"hive_metastore.gb_gold_schema.{name.replace('_df', '_table')}"  # Replace _df with _table in the name
    df.write.mode("overwrite").saveAsTable(table_name)
    print(f"Saved DataFrame {name} to {table_name}")